In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj3.ipynb")

# Project 3: Data Transformation

## Due Date: Friday, April 4, 5:00 PM

## Assignment Details

In this project, we'll be working with one month of data from sensors in buildings at UC Berkeley. This is a very typical real-world dataset—i.e. it's kind of a mess. The full dataset contains a giant `data` table of many billions of sensor readings over the course of a decade; we will look at a single month of that data. It also contains a variety of other tables that contextualize the readings.

Occasionally people believe that when the data has a well-structured schema, then it's simple to proceed! *We'll put this assumption to the test.*

Citation: Luo, N., Wang, Z., Blum, D. et al. A three-year dataset supporting research on building energy management and occupancy analytics. _Sci Data_ 9, 156 (2022). https://doi.org/10.1038/s41597-022-01257-x 

## Table Descriptions

* `buildings_site_mapping` - maps a `site` (full name) to a `building` (shortened name)
* `real_estate_metadata` - metadata about a unique piece of real estate
    * Note that `building` is more of a "building ID" and `building_name` matches `buildings_site_mapping.building`
* `data` - each row represents a sensor reading from a building at a particular time
* `metadata` - metadata about each sensor
* `uc_locations` - a standardized lookup table of UC location names (see Q3e)
* `ontology` - each row describes a relationship (aka `predicate`) between a `subject` and `object` (see Q5)
* `mapping` - each row maps the `rawname` of a sensor to the specific Brick ontology `Sensor` class (see Q5c)

## ER Diagram and Schema

The **ER diagram and schema for the database** is shown below.

* Each line represents a relationship between the two fields.
* The side of the line diverging to three lines / arrows represents the **"many"** side of the relationship, while the side of the line converging to one arrow represents the **"one"** side of the relationship.
* This file is available as `data/schema.png`.

<img src="data/schema.png">

If you make any queries to explore the data, do not forget to add a `LIMIT` clause; `LIMIT 10` is a good default to always add to the end of your queries. Otherwise, your connection may close as a result of trying to load excessively many rows.

In addition, for the entirety of the project, you may **(and should!)** make as many CTEs as you'd like. For questions where the output is just 1 row, you are not allowed to hardcode the answer. We reserve the right to penalize any hardcoded submissions.

## Logistics & Scoring Breakdown

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines. Free-response questions (marked 'm' in the table below) are manually graded.

Question | Points
--- | ---
1a	| m: 1
1b  | 1
1c	| 1
1d	| m: 2
2a	| 3
2b	| 1
3a	| m: 1
3b	| 1
3c	| 1
3d  | m: 1
3e  | 2
4a	| 2
4b	| 2
4c	| 3
5a  | 1
5b  | 1
5c  | 3
**Total** | 27

**Grand Total:** 27 points (autograded: 22, manual: 5)

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

In [3]:
# Set up autoreloading imported .py modules such as data101_utils.py
%load_ext autoreload
%autoreload 2

## Loading Up the Database
To load the database, run the following cell.

In [4]:
!psql postgresql://jovyan@127.0.0.1:5432/ucb_buildings -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'
!psql postgresql://jovyan@127.0.0.1:5432/postgres -c 'DROP DATABASE IF EXISTS ucb_buildings'
!psql postgresql://jovyan@127.0.0.1:5432/postgres -c 'CREATE DATABASE ucb_buildings'
# !psql -h localhost -d ucb_buildings -f ~/_shared/data101-readwrite/proj3_data/buildings.sql # Dev path
!psql -h localhost -d ucb_buildings -f ../../../_shared/data101-readonly/proj3_data/buildings.sql # Student Path

 pg_terminate_backend 
----------------------
(0 rows)

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
CREATE EXTENSION
COMMENT
CREATE EXTENSION
COMMENT
CREATE FUNCTION
ALTER FUNCTION
CREATE AGGREGATE
ALTER AGGREGATE
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE MATERIALIZED VIEW
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 92
COPY 102386
COPY 100000
COPY 381
COPY 9509
COPY 26183
COPY 5276
COPY 12
ALTER TABLE
ALTER TABLE
ALTER TABLE
REFRESH MATERIALIZED VIEW


Now, run the following cell to connect to the `ucb_buildings` database. There should be no errors after running the following cells.

In [51]:
# Just run the following cell, no further action is needed.
from data101_utils import GradingUtil
grading_util = GradingUtil("proj3")
grading_util.prepare_autograder("ucb_buildings")

Successfully opened database connection


In [52]:
# Fetch random rows to check table exists
grading_util.run_sql("SELECT * FROM data LIMIT 10;")

,time,id,value
0,2018-06-12 23:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.74
1,2018-06-12 22:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.51
2,2018-06-12 22:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.27
3,2018-06-12 22:15:10+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.02
4,2018-06-12 22:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.77
5,2018-06-12 21:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.54
6,2018-06-12 21:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.30
7,2018-06-12 21:15:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.04
8,2018-06-12 21:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65180.80
9,2018-06-12 20:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65180.57


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## New Query Execution Workflow (you MUST read this!)

In previous projects, we used an extension called JupySQL to run SQL directly in your Jupyter notebook. However, this can cause the notebook to be very long, and it also introduces unnecessary complexity when running the autograder. Starting with this project, we will be moving to a new query execution workflow where you will write your SQL queries in separate `.sql` files under the `queries` directory. It goes like this:

1. Read and understand the question
2. Open the `.sql` file that corresponds to that question. For example, for Q1b, you should open `queries/1b.sql`
3. Write your SQL code in that file. **If there is starter code, make sure you do NOT delete any of it otherwise the autograder will fail!**
4. **Save your SQL file.** This is ***SUPER IMPORTANT*** so that when you test your query result, it is running your most recent query.
5. Go back to your project Jupyter notebook, and run the cell that contains the call to `grading_util.run_file`. It will look something like this:
```python
result_1b = grading_util.run_file("1b")
result_1b
```
6. Observe the output and run tests

If you want to run one-off SQL queries (for example to explore the database), you can do that using `grading_util.run_sql` like so:
```python
grading_util.run_sql("SELECT 'YOUR CODE HERE';")
```
7. Once you're done with the project, upload your `queries.zip` file (instead of `results.zip`). This file will be automatically created when you run `grading_util.prepare_submission_and_cleanup()`.

**Dos and Don'ts**

- If you encounter an error, **read the entire error message** before asking for help - it might be long, but it can be helpful! For example, if you have a syntax error in your SQL code, it should tell you which line it occurred on and where on that line it occurred.
- You may find it useful to use **JupyterHub's split screen feature** so you can have the notebook and SQL file side by side. You can use this by dragging the tabs at the top. You can either do a left/right split or top/bottom split.
- If you need to run a SQL statement that spans multiple lines, **use Python's multiline strings** which are enclosed in triple quotes:
```python
grading_util.run_sql("""
SELECT 'YOUR CODE HERE';
SELECT 'YOUR CODE HERE';
SELECT 'YOUR CODE HERE';
""")
```
- Make sure to **terminate all SQL statements with a semicolon** and **enclose all subqueries in parentheses**.
- You can run multiple statements within a file or within a single `run_sql` call. **The results of each statement will be returned in a list; you may index into the list to view the results of an individual statement.** If no rows are outputted by that statement, `None` will be returned (instead of a `pandas.DataFrame`)
- You can learn more about the behavior of `run_sql` and `run_file` by running the cells below to read their docstrings

In [53]:
?grading_util.run_sql

Signature: grading_util.run_sql(query, explain=False, explain_analyze=False)
Docstring:
Executes SQL statement(s) as a query string.

Args:
    query (str) - SQL statement(s) to execute. Semicolon can only be omitted
        if you are executing a single statement.
    explain (bool) - True if you want to prepend EXPLAIN to the query
    explain_analyze (bool) - True if you want to prepend EXPLAIN ANAYLZE to the query

Returns:
    If there was only 1 SQL statement, the return type will be either pandas.DataFrame (if the
    query resulted in a table with rows) or None otherwise.
        
    If there were multiple SQL statements, the return type will be a list
    where each element in the list is the output table of each statement as a pandas.DataFrame
    or None if the output had no rows.

    NOTE: An output of None can happen if SELECT ... WHERE ... filters out all rows
    or if you're creating a table/view/materialized view, for example.

Raises:
    ValueError if:
        quer

In [54]:
?grading_util.run_file

Signature:
grading_util.run_file(
    path_to_sql_file,
    explain=False,
    explain_analyze=False,
    use_queries_dir=True,
)
Docstring:
Runs the SQL statement(s) in the given SQL file.
If the .sql file extension is not provided, it is automatically added.

Args:
    path_to_sql_file (str) - path to SQL file you want to execute
    explain (bool) - See the docstring of GradingUtil.execute
    explain_analyze (bool) - See the docstring of GradingUtil.execute
    use_queries_dir (bool) - True if you want to prepend `self.queries_dir` to the file path. Default True.

Returns:
    See the docstring of GradingUtil.execute
File:      ~/sp25-materials.git/proj/proj3/data101_utils.py
Type:      method

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


<!-- BEGIN QUESTION -->

## Question 1: Unboxing the Data

### Question 1a

As mentioned above, we are working with just one month of data. In the full database (which we don't have access to), tables like the `data` table have billions of rows. What do you notice about the design of the database schema above that helps support the large amount of data and minimize redundancy? **Keep your response to at most two sentences.**

**Hint:** There is no need to examine any data here. What is a technique learned in lecture? Define that technique.

The database uses normalization, a method that divides data into numerous related tables to prevent duplication and assure consistency. This simplifies the management of billions of rows by lowering storage requirements while ensuring data integrity.


<!-- END QUESTION -->

<br><br>

---
### Question 1b

The diagram claims that `buildings_site_mapping` has a **many-to-many relationship** with `real_estate_metadata`. Let's validate that.

**YOUR TASK:** Do the following in your query:

1. Find the **distinct** values of `buildings_site_mapping.building` that match **multiple** tuples in `real_estate_metadata.building_name`.
2. For each such value of `buildings_site_mapping.building`, return the matches as JSON via `JSON_AGG(real_estate_metadata)`. (See [Table 9-49 in the Postgres documentation](https://www.postgresql.org/docs/9.5/functions-aggregate.html)).
3. **Order your final result by `buildings_site_mapping.building` ascending**.

**Hint:** You should use a CTE to find the distinct buildings of `buildings_site_mapping` before applying necessary table joins.

Your first row should look like this:

| building | json_agg |
| :--- | :--- |
| ALUMNI HOUSE | [{'location': 'BERKELEY', 'building': '1215', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE C22A2M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 15590, 'bldg_asf': 8719, 'levels': 2, 'year': '1954', 'owner': 'UC', 'mp_code': 'P', 'book_value': 106819}, {'location': 'IRVINE', 'building': '9207', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE CAMPUS', 'city_name': 'IRVINE', 'county': 'ORANGE', 'category': 'GENERAL', 'osfg': 4027, 'bldg_asf': 2549, 'levels': 1, 'year': '1984', 'owner': 'UC', 'mp_code': 'P', 'book_value': 41981}, {'location': 'FRANCISC SOAN', 'building': '2032', 'building_name': 'ALUMNI HOUSE', 'address': 'PARNASS US AVE', 'city_name': 'FRANCISC OSAN', 'county': 'FRANCISC OSAN', 'category': 'HEALTH SCIENCE', 'osfg': 7217, 'bldg_asf': 5079, 'levels': 3, 'year': '1915', 'owner': 'UP', 'mp_code': 'P', 'book_value': 135923}] |

This is because `ALUMNI HOUSE` is a buildling name in both `buildings_site_mapping` and `real_estate_metadata`. In `real_estate_metadata`, these rows below have `buildling_name = 'ALUMNI HOUSE'` (see the query result below). All of the metadata from these 3 rows is incorporated in the `JSON_AGG` result (each row is an element in the JSON array surrounded by curly braces).

In [55]:
grading_util.run_sql("""
SELECT *
FROM real_estate_metadata
WHERE building_name = 'ALUMNI HOUSE';
""")


,location,building,building_name,address,city_name,county,category,osfg,bldg_asf,levels,year,owner,mp_code,book_value
0,BERKELEY,1215,ALUMNI HOUSE,CORE C22A2M0PUS,BERKELEY,ALAMEDA,GENERAL,15590,8719,2,1954,UC,P,106819.0
1,IRVINE,9207,ALUMNI HOUSE,CORE CAMPUS,IRVINE,ORANGE,GENERAL,4027,2549,1,1984,UC,P,41981.0
2,FRANCISC SOAN,2032,ALUMNI HOUSE,PARNASS US AVE,FRANCISC OSAN,FRANCISC OSAN,HEALTH SCIENCE,7217,5079,3,1915,UP,P,135923.0


In [56]:
result_1b = grading_util.run_file("1b")
grading_util.save_results("result_1b", result_1b)
result_1b

,building,json_agg
0,ALUMNI HOUSE,"[{'location': 'BERKELEY', 'building': '1215', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE C22A2M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 15590, 'bldg_asf': 8719, 'levels': 2, 'year': '1954', 'owner': 'UC', 'mp_code': 'P', 'book_value': 106819}, {'location': 'FRANCISC SOAN', 'building': '2032', 'building_name': 'ALUMNI HOUSE', 'address': 'PARNASS US AVE', 'city_name': 'FRANCISC OSAN', 'county': 'FRANCISC OSAN', 'category': 'HEALTH SCIENCE', 'osfg': 7217, 'bldg_asf': 5079, 'levels': 3, 'year': '1915', 'owner': 'UP', 'mp_code': 'P', 'book_value': 135923}, {'location': 'IRVINE', 'building': '9207', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE CAMPUS', 'city_name': 'IRVINE', 'county': 'ORANGE', 'category': 'GENERAL', 'osfg': 4027, 'bldg_asf': 2549, 'levels': 1, 'year': '1984', 'owner': 'UC', 'mp_code': 'P', 'book_value': 41981}]"
1,CAMPBELL,"[{'location': 'LOS ANGELES', 'building': '4294', 'building_name': 'CAMPBELL', 'address': 'PORTOLA P11L8A1Z1A', 'city_name': 'LOS ANGELES', 'county': 'LOS ANGELES', 'category': 'GENERAL', 'osfg': 55401, 'bldg_asf': 32619, 'levels': 5, 'year': '1954', 'owner': 'UC', 'mp_code': 'P', 'book_value': 6247192.0}, {'location': 'BERKELEY', 'building': '1027', 'building_name': 'CAMPBELL', 'address': 'CORE C28A5M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 83373, 'bldg_asf': 50745, 'levels': 7, 'year': '2014', 'owner': 'UC', 'mp_code': 'P', 'book_value': 75256344.0}]"
2,HERTZ,"[{'location': 'DAVIS', 'building': '9454', 'building_name': 'HERTZ', 'address': '7000 EAST 2A4V0E0 LAKE', 'city_name': 'LIVERMOR E', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 10904, 'bldg_asf': 8183, 'levels': 1, 'year': '1976', 'owner': 'UC', 'mp_code': 'P', 'book_value': 0}, {'location': 'BERKELEY', 'building': '1423', 'building_name': 'HERTZ', 'address': 'CORE CAMPUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 31362, 'bldg_asf': 13518, 'levels': 4, 'year': '1958', 'owner': 'UC', 'mp_code': 'P', 'book_value': 3491181.0}]"
3,SOUTH,"[{'location': 'DAVIS', 'building': '3275', 'building_name': 'SOUTH', 'address': 'CORE CAMPUS', 'city_name': 'DAVIS (YOLO)', 'county': 'YOLO', 'category': 'GENERAL', 'osfg': 26594, 'bldg_asf': 15323, 'levels': 4, 'year': '1912', 'owner': 'UC', 'mp_code': 'P', 'book_value': 3585399.0}, {'location': 'BERKELEY', 'building': '1484', 'building_name': 'SOUTH', 'address': 'CORE CAMPUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 30401, 'bldg_asf': 15473, 'levels': 4, 'year': '1873', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1426174.0}]"
4,SPROUL,"[{'location': 'DAVIS', 'building': '3815', 'building_name': 'SPROUL', 'address': 'CORE CAMPUS', 'city_name': 'DAVIS (YOLO)', 'county': 'YOLO', 'category': 'GENERAL', 'osfg': 53874, 'bldg_asf': 27913, 'levels': 10, 'year': '1963', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1620880.0}, {'location': 'BERKELEY', 'building': '1210', 'building_name': 'SPROUL', 'address': 'CORE 2C6A0M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 111197, 'bldg_asf': 66680, 'levels': 6, 'year': '1941', 'owner': 'UC', 'mp_code': 'P', 'book_value': 4278819.0}, {'location': 'RIVERSIDE', 'building': 'P5523', 'building_name': 'SPROUL', 'address': 'CORE CAMPUS', 'city_name': 'RIVERSIDE', 'county': 'RIVERSIDE', 'category': 'GENERAL', 'osfg': 80988, 'bldg_asf': 45532, 'levels': 5, 'year': '1965', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1296147.0}]"


In [57]:
result_1b = grading_util.load_results("result_1b")[0]

In [58]:
grader.check("q1b")

q1b results: All test cases passed!

<br><br>

---
### Question 1c

Now find examples of many matches in the opposite direction:

1. For each **distinct** `real_estate_metadata.building_name` value, find the ones that have **multiple matches** in `buildings_site_mapping.building`.
2. For each `building_name` value, return a `JSON_AGG` of the matches in `buildings_site_mapping`.
3. **Order your final result by `building_name` in ascending order.**

**Hint:** You should use a CTE to find the distinct building names of `real_estate_metadata` before applying necessary table joins.

Your table header should look like this:

| building_name | json_agg |
| :--- | :--- |

In [59]:
result_1c = grading_util.run_file("1c")
grading_util.save_results("result_1c", result_1c)
result_1c

,building_name,json_agg
0,EDWARDS FLD,"[{'site': 'Edwards Stadium West', 'building': 'EDWARDS FLD'}, {'site': 'Edwards Stadium East', 'building': 'EDWARDS FLD'}]"
1,FAC CLUB,"[{'site': 'Womens Faculty Club', 'building': 'FAC CLUB'}, {'site': 'Men's Faculty Club', 'building': 'FAC CLUB'}]"
2,HAAS STU BLD,"[{'site': 'Haas School of Business', 'building': 'HAAS STU BLD'}, {'site': 'Haas School Student Services (Water)', 'building': 'HAAS STU BLD'}]"
3,SIMON,"[{'site': 'Boalt and Simon Hall', 'building': 'SIMON'}, {'site': 'Simpson Center', 'building': 'SIMON'}]"


In [60]:
result_1c = grading_util.load_results("result_1c")[0]

In [61]:
grader.check("q1c")

q1c results: All test cases passed!

<!-- BEGIN QUESTION -->

<br><br>

---
### Question 1d

Address the two questions below:

1. Can you uniquely determine the building given the sensor data? Why? (**Hint:** given a row in the `data` table, can you determine a **uniquely** associated row in `real_estate_metadata` table? Your answer should draw insights from 1b.)
2. Could `buildings_site_mapping.building` be a valid foreign key pointing to `real_estate_metadata.building_name`? (**Hint:** think about what kinds of columns can be a foreign key.)

Please keep your response to **exactly 1 sentence for each subpart and format your answer like so:**

1. YOUR ANSWER
2. YOUR ANSWER

No, you cannot determine the building uniquely from a row in the data table since numerous buildings might have the same building name in real_estate_metadata, as seen in the JSON aggregate of Question 1b.

No, buildings_site_mapping.building is not an acceptable foreign key for real_estate_metadata.building_name because building names are not unique in the metadata, and foreign keys must reference unique values.

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 2: Looking for Outliers in the Readings
Physical sensors, such as the ones responsible for generating this data, are notorious for occasionally producing crazy outliers. In this section, we will undergo some data cleaning to address these outliers.

The readings from all different types of sensors are mixed together within this `data` table. This assortment of mixed readings will require some extra work to identify the outliers. Let's get started.

### Question 2a: Outlier Detection

Let's start with finding the outliers in the `data` table. We'll define an outlier as an observation that is more than **3 Hampel X84 intervals** away from the median of all values from the same sensor ID. **You must use `1.4826` as one Hampel X84 interval.** See lectures on outliers for more information.

Create a view `labeled_data` that contains all of the columns in `data` and adds 3 additional columns on the far right:

1. `median`: the median of `value` for that sensor `id` using `PERCENTILE_DISC`. See [Table 9-51 on the Postgres docs](https://www.postgresql.org/docs/9.4/functions-aggregate.html#FUNCTIONS-ORDEREDSET-TABLE)
3. `mad`: the Median Absolute Deviation (MAD) of the `value` column for that sensor `id`
4. `is_outlier`: is `TRUE` if that `value` is an outlier as defined above, and `FALSE` otherwise.
    - **Also, for data points where the `mad` is 0, set this to `FALSE`.**

Your table header should look like this:

| time | id | value | median | mad | is_outlier |
| :--- | :--- | :--- | :--- | :--- | :--- |

**Hints:**

- You should use CTEs to first compute the median and MAD values. Feel free to create as many CTEs as you need to complete the objective. Our staff solution used 3:
    - One to compute the median of *each* sensor id
    - One to compute the absolute differences for each data point relative to its sensor id median
    - One to finally compute the median of these absolute differences.
- You may find the `CASE` statement useful. See the [course notes](https://data101.org/notes/1-SQL/review.html#case).

In [62]:
result_2a = grading_util.run_file("2a")[2]
grading_util.save_results("result_2a", result_2a)
result_2a

,time,id,value,median,mad,is_outlier
0,2018-06-07 00:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,812.6,727.500,16.200,True
1,2018-06-07 00:00:00+00:00,a51d3bed-a8b6-5321-96a1-c5542caca70c,75.0,6.186,4.957,True
2,2018-06-07 00:00:00+00:00,a6336afc-cdb7-5f87-a4bf-67411ea1b5cc,54.0,32.000,1.000,True
3,2018-06-07 00:00:00+00:00,a652779f-8433-5ad3-9310-f4f1062e4d04,14.0,8.000,1.000,True
4,2018-06-07 00:00:00+00:00,a760a23a-8de4-5ce3-939a-826bf31f01df,109.0,69.000,7.000,True
...,...,...,...,...,...,...
95,2018-06-07 00:00:00+00:00,a7145dcc-90e5-5bfb-9b1c-277615779c16,4.0,3.000,1.000,False
96,2018-06-07 00:00:00+00:00,a71d821d-d6e6-5ee9-86fa-2b075f13c2cc,99.0,81.000,17.000,False
97,2018-06-07 00:00:00+00:00,a71fbdfd-26ad-562c-b193-9dc414b851f0,122.9,58.700,35.700,False
98,2018-06-07 00:00:00+00:00,a744b292-f4cf-5f85-9e4a-5e626e19d5af,107.0,74.000,18.000,False


In [63]:
result_2a = grading_util.load_results("result_2a")[0]

In [64]:
grader.check("q2a")

q2a results: All test cases passed!

<br><br>

---

### Question 2b: Outlier Handling (Winsorization)

In this step, define a view `cleaned_data` containing all the columns of `labeled_data` and one additional column on the far right called `clean_value`. This column will contain a copy of `labeled_data.value` if that value is not an outlier. For outliers, it should contain the value Winsorized to the nearest outlier boundary value (3 Hampel X84 intervals from the median). **If the MAD is 0, then the cleaned value should be unchanged (i.e., the same as the original value).**

To reiterate, we consider a value to be an outlier if it is more than (strictly greater than or strictly less than) 3 Hampel X84 intervals from the median. **You must use `1.4826` once again as one Hampel X84 interval.**

Your table header should look like this:

| time | id | value | median | mad | is_outlier | clean_value |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- |

In [65]:
result_2b = grading_util.run_file("2b")[2]
grading_util.save_results("result_2b", result_2b)
result_2b

,time,id,value,median,mad,is_outlier,clean_value
0,2018-06-07 00:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,812.6,727.500,16.200,True,799.554360
1,2018-06-07 00:00:00+00:00,a51d3bed-a8b6-5321-96a1-c5542caca70c,75.0,6.186,4.957,True,28.233745
2,2018-06-07 00:00:00+00:00,a6336afc-cdb7-5f87-a4bf-67411ea1b5cc,54.0,32.000,1.000,True,36.447800
3,2018-06-07 00:00:00+00:00,a652779f-8433-5ad3-9310-f4f1062e4d04,14.0,8.000,1.000,True,12.447800
4,2018-06-07 00:00:00+00:00,a760a23a-8de4-5ce3-939a-826bf31f01df,109.0,69.000,7.000,True,100.134600
...,...,...,...,...,...,...,...
95,2018-06-07 00:00:00+00:00,a7145dcc-90e5-5bfb-9b1c-277615779c16,4.0,3.000,1.000,False,4.000000
96,2018-06-07 00:00:00+00:00,a71d821d-d6e6-5ee9-86fa-2b075f13c2cc,99.0,81.000,17.000,False,99.000000
97,2018-06-07 00:00:00+00:00,a71fbdfd-26ad-562c-b193-9dc414b851f0,122.9,58.700,35.700,False,122.900000
98,2018-06-07 00:00:00+00:00,a744b292-f4cf-5f85-9e4a-5e626e19d5af,107.0,74.000,18.000,False,107.000000


In [66]:
result_2b = grading_util.load_results("result_2b")[0]

In [67]:
grader.check("q2b")

q2b results: All test cases passed!

<!-- BEGIN QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 3: Entity Resolution

### Question 3a

There is a lot of mess in this dataset related to entity names. As a start, have a look at all of the distinct values in the `units` field of the `metadata` table, which contains the units of measurement for a particular piece of metadata (you can use the ungraded code cell below or the terminal).

If you are unfamiliar with a unit of measurement, try searching for it and its abbreviation online.

What do you notice about these values? Are there any duplicates? **Limit your response to one sentence.**

Many duplication are generated by uneven formatting, such as changes in capitalization, spacing, and nomenclature for the same units (e.g., "F", "°F", and "Fahrenheit")

In [68]:
grading_util.run_sql("""
SELECT DISTINCT units
FROM metadata
ORDER BY units;
""")

,units
0,A
1,Amps
2,Bottom
3,CF
4,CFm
...,...
29,uS
30,V
31,Volts
32,W


<!-- END QUESTION -->

<br><br>

---

### Question 3b

Sometimes entity resolution is as simple as a text transformation. For example, how many unique `units` values are there, and how many would there be if we ignored case (upper vs. lower case)? Your output should be a table with one row and two columns:

1. `num_unique_units`: The number of unique `units` values (case sensitive)
2. `num_unique_units_ignore_case`: The number of unique `units` values if we ignored case

Your table header should look like this:

| num_unique_units | num_unique_units_ignore_case |
| :--- | :--- |

In [69]:
result_3b = grading_util.run_file("3b")
grading_util.save_results("result_3b", result_3b)
result_3b

,num_unique_units,num_unique_units_ignore_case
0,34,29


In [70]:
result_3b = grading_util.load_results("result_3b")[0]

In [71]:
grader.check("q3b")

q3b results: All test cases passed!

<br><br>

---

### Question 3c

Arguably, we shouldn't care about these alternative unit labels, *as long as each sensor in the same **`class`** uses a single unique value of `units`*. After all, maybe the capitalization means something to somebody!

Write a SQL query that returns a table containing **1 row and 1 column** of value `TRUE` if the condition (in italics above) holds, or `FALSE` otherwise. This column should be called `are_units_consistent`.

**You must use the `ALL` operator ([Postgres docs](https://www.postgresql.org/docs/current/functions-comparisons.html#FUNCTIONS-COMPARISONS-ALL)).** Example usage: `<value> = ALL (SELECT <column or columns> FROM <table>)`

It may also be helpful to write a CTE.

Your table header should look like this:

| are_units_consistent |
| :--- |

In [72]:
result_3c = grading_util.run_file("3c")
grading_util.save_results("result_3c", result_3c)
result_3c

,are_units_consistent
0,True


In [73]:
result_3c = grading_util.load_results("result_3c")[0]

In [74]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

<br><br>

---

### Question 3d

Moving on, have a look at the `real_estate_metadata` table—starting with the distinct values in the `location` field! What do you notice about the spelling of some of these values? (If you're unfamiliar with these locations, search them up online.) **Keep your response to at most 1 sentence.**


Certain locations contain conflicting or inaccurate spellings, such as "PARANNSS AVE" instead of "PARNASSUS AVE," and differing abbreviations, such as "AVE" vs "AV"

<!-- END QUESTION -->

<br><br>

---
### Question 3e

It turns out this `real_estate_metadata` table is the result of an [OCR scan](https://en.wikipedia.org/wiki/Optical_character_recognition), which can sometimes be inaccurate. We will just focus on cleaning up the `location` column for the time being, and leave you to imagine the effort required to do a full cleanup of all columns.

We also created a lookup table of standardized names, `uc_locations`. You can treat these as "ground truth" correct spellings of locations found in `real_estate_metadata`.

**YOUR TASK:** Write a SQL query that returns the columns `building_name, address, location, clean_location` where `clean_location` contains the best match from `uc_locations.loc_name`. There are several ways you can go about doing this, and we encourage you to try different approaches:

1. **Method 1:** For each distinct `real_estate_metadata.location` that is misspelled, determine the correct spelling in `uc_locations.loc_name` and use `CASE` to assign the correct spelling.
2. **Method 2:** Use functions from a Postgres extension package (which we have preloaded for you) like `fuzzystrmatch` (for [fuzzy string matching](https://en.wikipedia.org/wiki/Approximate_string_matching)) or `pg_trgm` (for [trigram](https://en.wikipedia.org/wiki/N-gram) matching). You can use any of the string functions in those packages if you'd like ([fuzzystrmatch docs](https://www.postgresql.org/docs/current/fuzzystrmatch.html) or [pg_trgm docs](https://www.postgresql.org/docs/current/pgtrgm.html)). Examples:
    - `word_similarity` (course staff has found the most success using this one)
    - `metaphone`
    - `levenshtein`
    - **NOTE**: Depending on which function you use, you may still need to include logic to handle edge cases (e.g. use `CASE` for certain mispelled locations that don't match up well with the correct spelling)

You can choose to do this question in whatever manner you wish as long as your query does **not** use `CREATE TABLE`, `INSERT INTO`, or `UPDATE`.

Your table header should look like this:

| building_name | address | location | clean_location |
| :--- | :--- | :--- | :--- |

In [75]:
result_3e = grading_util.run_file("3e")
grading_util.save_results("result_3e", result_3e)
result_3e

,building_name,address,location,clean_location
0,CLEARY,2424 CHANNING,BERKELEY,BERKELEY
1,WALNUT19 25,1925 WALNUT,BERKELEY,BERKELEY
2,ERCHLDE DCNTR,2339 HASTE,BERKELEY,BERKELEY
3,MRI MODLR II,CORE 2C1A5M4P-6U0S,BERKELEY,BERKELEY
4,UNIV 2154- 60,UNIVERSIT 2Y55,BERKELEY,BERKELEY
...,...,...,...,...
5271,WS 314 HOUSE,OAKLAND 1A7V3E53 W,AG FIELD STAT,AG FIELD STAT
5272,WS 315 HOUSE,OAKLAND 1A7V3E53 W,AG FIELD STAT,AG FIELD STAT
5273,WS 316 HOUSE,OAKLAND 1A7V3E53 W,AG FIELD STAT,AG FIELD STAT
5274,WS 317 WHOSUSE,OAKLAND A17V3E53 W,AG FIELD STAT,AG FIELD STAT


In [76]:
result_3e = grading_util.load_results("result_3e")[0]

In [77]:
grader.check("q3e")

q3e results: All test cases passed!

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 4: Interpolating Missing Data
Real-world data, real-world problems. Our sensors should be reporting every 15 minutes, but you can be sure that we're missing some data. Here, we will fix it. It's a bit more involved than what we looked at in class!

### Question 4a: Finding Missing Readings
In the `data` table, the `id` column identifies a unique sensor. Sensor readings should be recorded every 15 minutes from every sensor. Are we missing any readings, and if so, which ones? We will focus on readings **from the same sensor** that are separated by at least 30 minutes or more; readings that are 0 (inclusive) to 30 (exclusive) minutes apart are considered to be fine.

To answer this question you'll need to read up a bit on [SQL timestamps](https://www.postgresql.org/docs/current/datatype-datetime.html) and [functions for manipulating datetime types](https://www.postgresql.org/docs/current/functions-datetime.html). Taking the time now to look at the documentation will save you time later on. Have a particular look at the following:
- The [DATE_TRUNC](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC) function will quantize times to the nearest unit of your choosing. For example, to round the `time` field to the nearest minute you can say `DATE_TRUNC('minute', time)`. **You'll need to quantize to minutes right away before you worry about missing readings.**
- There are various ways to enter constant intervals of time as strings. For example, a 30-minute interval can be written as `INTERVAL '30 minutes'` or `'30 minutes'::INTERVAL`. See [datetime interval inputs](https://www.postgresql.org/docs/current/datatype-datetime.html#DATATYPE-INTERVAL-INPUT) for more info.
- You can do arithmetic on datetime types [as documented here](https://www.postgresql.org/docs/current/functions-datetime.html#OPERATORS-DATETIME-TABLE). That will handle all the weird periodicities of clocks and calendars for you. **Pay attention to the input and output types of these functions! Also double check the data types for the columns of the `data` table!**
- You will need to use the [LAG](https://www.postgresql.org/docs/current/functions-window.html) window function.

Create a view called `gaps` that augments the `data` schema with three columns:

- `lag_time` is the quantized time of the previous reading for that sensor (relative to the current row for a particular row)
- `lag_value` is the value of the previous reading for that sensor
- `time_diff` is the difference in quantized time between this reading and the previous reading

**The view should only contain rows where `time_diff` is greater than or equal to 30 minutes**.

Your view's column headers should look like this:

| id | time | value | lag_time | lag_value | time_diff |
| :--- | :--- | :--- | :--- | :--- | :--- |

In [78]:
result_4a = grading_util.run_file("4a")[2]
grading_util.save_results("result_4a", result_4a)
result_4a

,id,time,value,lag_time,lag_value,time_diff
0,a3d47b1a-985e-5395-a6ee-719dad9b580f,2018-06-13 09:15:00+00:00,0.056,2018-06-13 08:45:00+00:00,0.056,0 days 00:30:00
1,a3e8e405-0eed-59b6-8747-fe892a6f93de,2018-06-09 04:15:00+00:00,280.800,2018-06-09 03:45:00+00:00,281.600,0 days 00:30:00
2,a46fb790-028c-5d17-a3e6-7d08daec0c03,2018-06-13 09:15:00+00:00,281.460,2018-06-13 08:45:00+00:00,281.390,0 days 00:30:00
3,a470ac6d-d448-522b-9b74-8ed27f24de25,2018-06-09 04:15:00+00:00,0.004,2018-06-09 03:45:00+00:00,0.006,0 days 00:30:00
4,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 00:45:00+00:00,27.273,2018-06-07 00:15:00+00:00,30.000,0 days 00:30:00
...,...,...,...,...,...,...
766,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 08:15:00+00:00,5.400,2018-06-13 07:45:00+00:00,0.012,0 days 00:30:00
767,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 13:00:00+00:00,5.400,2018-06-13 08:15:00+00:00,5.400,0 days 04:45:00
768,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 14:30:00+00:00,2.700,2018-06-13 13:00:00+00:00,5.400,0 days 01:30:00
769,a8d640b1-c17f-51cf-b06f-de6e704e87c1,2018-06-13 09:15:00+00:00,106.000,2018-06-13 08:45:00+00:00,106.000,0 days 00:30:00


In [79]:
result_4a = grading_util.load_results("result_4a")[0]

In [80]:
grader.check("q4a")

q4a results: All test cases passed!

### Question 4b: Creating tuples for missing readings
Now, we need to manufacture new tuples to fill in the gaps. For example, if you had one tuple with an `id` of `abc` timestamped at 1:00 PM and the next tuple with an `id` of `abc` timestamped at 1:45 PM, you'll need to manufacture two new tuples containing both an `id` of `abc` as well as `NULL` values — one tuple should be timestamped at 1:15 PM and the other at 1:30 PM. We will worry about replacing the `NULL` values in the next step.

To manufacture tuples not related to stored data in the database, we'll need to use a *table-valued function*. The table-valued function we want here is `GENERATE_SERIES` [(documented here)](https://www.postgresql.org/docs/current/functions-srf.html), which we will use to generate *and sequentially timestamp* the right number of tuples to match the number of tuples we found missing.

To get a feel for `GENERATE_SERIES`, consider the following simple query that generates a table of integers with intervals of size three between them. (This is similar to how the `range` function in Python works, except the `stop` parameter is inclusive.)

In [81]:
grading_util.run_sql("""
SELECT *
FROM GENERATE_SERIES(1, 10, 3);
""")

,generate_series
0,1
1,4
2,7
3,10


**YOUR TASK:** Create a view `complete` that contains the tuples from `data` as well as new tuples that fill in any gaps greater than or equal to 30 minutes. Each gap should be filled by adding tuples in increments of 15 minutes from the *start* of the gap, **with `NULL` as the fill value**. You probably want to use your `gaps` view (from Q4a) as well as `GENERATE_SERIES` to do this!

**Hints:** 
- The lower and upper bounds in `GENERATE_SERIES` (in pseudocode) should be `lag_time + 15 minutes` and `time - 15 minutes`, respectively.
- You may want to use `UNION ALL` to combine your the set of rows that were generated and the existing rows in `data`

Your table header should look like this:

| id | time | value |
| :--- | :--- | :--- |

In [82]:
grading_util.run_sql("""
SELECT * FROM gaps LIMIT 5;
""")



,id,time,value,lag_time,lag_value,time_diff
0,a3d47b1a-985e-5395-a6ee-719dad9b580f,2018-06-13 09:15:00+00:00,0.056,2018-06-13 08:45:00+00:00,0.056,0 days 00:30:00
1,a3e8e405-0eed-59b6-8747-fe892a6f93de,2018-06-09 04:15:00+00:00,280.800,2018-06-09 03:45:00+00:00,281.600,0 days 00:30:00
2,a46fb790-028c-5d17-a3e6-7d08daec0c03,2018-06-13 09:15:00+00:00,281.460,2018-06-13 08:45:00+00:00,281.390,0 days 00:30:00
3,a470ac6d-d448-522b-9b74-8ed27f24de25,2018-06-09 04:15:00+00:00,0.004,2018-06-09 03:45:00+00:00,0.006,0 days 00:30:00
4,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 00:45:00+00:00,27.273,2018-06-07 00:15:00+00:00,30.000,0 days 00:30:00


In [83]:
result_4b = grading_util.run_file("4b")
result_4b


[None,
                                       id                      time     value
 0   a3d3326f-20ab-5f1d-97c7-f3084df43f06 2018-06-07 00:00:09+00:00  65085.99
 1   a3d3326f-20ab-5f1d-97c7-f3084df43f06 2018-06-07 00:15:09+00:00  65086.16
 2   a3d3326f-20ab-5f1d-97c7-f3084df43f06 2018-06-07 00:30:09+00:00  65086.35
 3   a3d3326f-20ab-5f1d-97c7-f3084df43f06 2018-06-07 00:45:09+00:00  65086.52
 4   a3d3326f-20ab-5f1d-97c7-f3084df43f06 2018-06-07 01:00:09+00:00  65086.71
 ..                                   ...                       ...       ...
 95  a3d3326f-20ab-5f1d-97c7-f3084df43f06 2018-06-07 23:45:09+00:00  65100.44
 96  a3d3326f-20ab-5f1d-97c7-f3084df43f06 2018-06-08 00:00:09+00:00  65100.63
 97  a3d3326f-20ab-5f1d-97c7-f3084df43f06 2018-06-08 00:15:09+00:00  65100.80
 98  a3d3326f-20ab-5f1d-97c7-f3084df43f06 2018-06-08 00:30:09+00:00  65100.97
 99  a3d3326f-20ab-5f1d-97c7-f3084df43f06 2018-06-08 00:45:09+00:00  65101.16
 
 [100 rows x 3 columns]]

In [84]:
result_4b = grading_util.run_file("4b")[1]
grading_util.save_results("result_4b", result_4b)
result_4b

,id,time,value
0,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:00:09+00:00,65085.99
1,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:15:09+00:00,65086.16
2,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:30:09+00:00,65086.35
3,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:45:09+00:00,65086.52
4,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:00:09+00:00,65086.71
...,...,...,...
95,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 23:45:09+00:00,65100.44
96,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-08 00:00:09+00:00,65100.63
97,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-08 00:15:09+00:00,65100.80
98,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-08 00:30:09+00:00,65100.97


In [85]:
result_4b = grading_util.load_results("result_4b")[0]

In [86]:
grader.check("q4b")

q4b results: All test cases passed!

### Question 4c: Linear Interpolation
*Note: If you struggled with the previous subparts of this problem, you can use our table `complete_provided` instead of your `complete` view in this subpart.*

Now, given the `complete` view or the `complete_provided` table, your remaining task is to perform linear interpolation to fill in the missing values we manufactured in Q4b. We have code from the Data Preparation IV lecture that we can adapt from and use here! In particular, your database already includes the UDA (User-Defined Aggregate) `COALESCE_AGG` that we used in [lecture](https://docs.google.com/presentation/d/1s9FF9FrkQ4gu_1fg8UtaY-58FMvpkKJ-801bYly_1M8/edit#slide=id.g33c915dd0c9_0_3) (you can use it directly, there's no need to redefine it).

Below is an example usage of `COALESCE_AGG`, which is used to create `run_start`.

In [87]:
coalesce_agg_result = grading_util.run_sql("""
CREATE TABLE temp (
    id INTEGER,
    num INTEGER
);

INSERT INTO temp VALUES
    (1, 1),
    (2, 2),
    (3, NULL),
    (4, NULL),
    (5, 3),
    (6, NULL),
    (7, 4);

SELECT COALESCE_AGG(num) OVER (ORDER BY id) AS run_start FROM temp;

DROP TABLE temp;
""")[2]
coalesce_agg_result

,run_start
0,1
1,2
2,2
3,2
4,3
5,3
6,4


But note that in the lecture example of linear interpolation, we had a field that was used to order *all* the records in the table. In contrast, the ordering we care about here is the series of `time` for each sensor `id` *independently*. **You will need to make minor changes to adapt the linear interpolation code from class to work here.**

Your overall task across the next 3 subparts is to create a view `likely_data` that contains all the tuples from `complete`, with an additional column called `interpolated` that contains a copy of `value` if it is not `NULL`, and otherwise an interpolated value based on linear interpolation **per sensor `id` over time**.

To help you accomplish this in parts, we have broken it down for each subpart:

1. Q4ci is the forward pass
2. Q4cii is the backward pass
3. Q4ciii is the final actual interpolation

You may reference the [3-pass demo from lecture](https://docs.google.com/presentation/d/1s9FF9FrkQ4gu_1fg8UtaY-58FMvpkKJ-801bYly_1M8/edit#slide=id.g28ba83fdb0d_0_137).

#### Question 4ci: Forward Pass

Perform the forward pass by creating a view called `forward`. Your table header should look like:

| id | time | value | run | run_start | next_val |
| :--- | :--- | :--- | :--- | :--- | :--- |

**NOTE**: There are no hidden tests for this subpart, but they are not comprehensive. We will comprehensively test your final output in 4ciii.

In [88]:
grading_util.run_sql("""
SELECT COUNT(*) AS total_rows,
       COUNT(*) FILTER (WHERE value IS NULL) AS null_rows
FROM complete;
""")


,total_rows,null_rows
0,102386,2386


In [89]:
grading_util.run_sql("""
SELECT *
FROM complete
WHERE value IS NULL
LIMIT 10;
""")


,id,time,value
0,a3d47b1a-985e-5395-a6ee-719dad9b580f,2018-06-13 09:00:00+00:00,None
1,a3e8e405-0eed-59b6-8747-fe892a6f93de,2018-06-09 04:00:00+00:00,None
2,a46fb790-028c-5d17-a3e6-7d08daec0c03,2018-06-13 09:00:00+00:00,None
3,a470ac6d-d448-522b-9b74-8ed27f24de25,2018-06-09 04:00:00+00:00,None
4,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 00:30:00+00:00,None
5,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 01:15:00+00:00,None
6,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 17:30:00+00:00,None
7,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 18:30:00+00:00,None
8,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 19:00:00+00:00,None
9,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 19:30:00+00:00,None


In [90]:
grading_util.run_sql("""
WITH base AS (
    SELECT *,
           CASE WHEN value IS NULL THEN 1 ELSE 0 END AS is_null
    FROM complete
),
runs AS (
    SELECT *,
           SUM(is_null) OVER (
               PARTITION BY id ORDER BY time
           ) AS run
    FROM base
)
SELECT *
FROM runs
WHERE value IS NULL
LIMIT 10;
""")


,id,time,value,is_null,run
0,a3d47b1a-985e-5395-a6ee-719dad9b580f,2018-06-13 09:00:00+00:00,None,1,1
1,a3e8e405-0eed-59b6-8747-fe892a6f93de,2018-06-09 04:00:00+00:00,None,1,1
2,a46fb790-028c-5d17-a3e6-7d08daec0c03,2018-06-13 09:00:00+00:00,None,1,1
3,a470ac6d-d448-522b-9b74-8ed27f24de25,2018-06-09 04:00:00+00:00,None,1,1
4,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 00:30:00+00:00,None,1,1
5,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 01:15:00+00:00,None,1,2
6,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 17:30:00+00:00,None,1,3
7,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 18:30:00+00:00,None,1,4
8,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 19:00:00+00:00,None,1,5
9,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 19:30:00+00:00,None,1,6


In [91]:
grading_util.run_sql("SELECT COUNT(*) FROM complete;")


,count
0,102386


In [93]:
result_4ci = grading_util.run_file("4ci")[2]
grading_util.save_results("result_4ci", result_4ci)
result_4ci

,id,time,value,run,run_start,next_val
0,a603047b-c231-57bf-be62-39063ea79189,2018-06-07 00:30:00+00:00,None,1,0.348,0.248
1,a58833d6-95c8-52ec-be0b-567a14350239,2018-06-07 01:15:00+00:00,None,1,0.049,0.031
2,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 00:30:00+00:00,None,2,30.000,27.273
3,a58833d6-95c8-52ec-be0b-567a14350239,2018-06-07 02:30:00+00:00,None,2,0.031,0.022
4,a58833d6-95c8-52ec-be0b-567a14350239,2018-06-07 03:30:00+00:00,None,3,0.022,0.017
...,...,...,...,...,...,...
95,a58833d6-95c8-52ec-be0b-567a14350239,2018-06-08 06:45:00+00:00,None,45,0.014,0.014
96,a62e60b7-fdc4-5334-ae13-d207247f8450,2018-06-07 14:00:00+00:00,None,45,0.000,0.000
97,a58833d6-95c8-52ec-be0b-567a14350239,2018-06-08 07:45:00+00:00,None,46,0.014,0.014
98,a51d3bed-a8b6-5321-96a1-c5542caca70c,2018-06-08 22:30:00+00:00,None,46,3.135,10.239


In [94]:
result_4ci = grading_util.load_results("result_4ci")[0]

In [95]:
grader.check("q4ci")

q4ci results: All test cases passed!

#### Question 4cii: Backward Pass

Perform the backward pass by creating a view called `backward`.

Your table header should look like:

| id | time | value | run | run_start | next_val | run_end | run_rank | run_size |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |

**NOTE**: There are no hidden tests for this subpart, but they are not comprehensive. We will comprehensively test your final output in 4ciii.

In [96]:
result_4cii = grading_util.run_file("4cii")[2]
grading_util.save_results("result_4cii", result_4cii)
result_4cii

,id,time,value,run,run_start,next_val,run_end,run_rank,run_size
0,a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 10:00:02+00:00,0.000,137,0.000,NaN,0.000,0,3
1,a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 10:15:00+00:00,NaN,137,0.000,NaN,0.000,1,3
2,a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 10:30:00+00:00,NaN,137,0.000,0.000,0.000,2,3
3,a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 19:45:03+00:00,0.000,174,0.000,NaN,0.000,0,3
4,a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 20:00:00+00:00,NaN,174,0.000,NaN,0.000,1,3
...,...,...,...,...,...,...,...,...,...
95,a51d3bed-a8b6-5321-96a1-c5542caca70c,2018-06-07 22:15:00+00:00,NaN,20,5.825,76.923,76.923,2,3
96,a51d3bed-a8b6-5321-96a1-c5542caca70c,2018-06-07 23:00:00+00:00,8.021,23,7.937,NaN,8.021,0,3
97,a51d3bed-a8b6-5321-96a1-c5542caca70c,2018-06-07 23:15:00+00:00,NaN,23,8.021,NaN,103.448,1,3
98,a51d3bed-a8b6-5321-96a1-c5542caca70c,2018-06-07 23:30:00+00:00,NaN,23,8.021,103.448,103.448,2,3


In [97]:
result_4cii = grading_util.load_results("result_4cii")[0]

In [98]:
grader.check("q4cii")

q4cii results: All test cases passed!

#### Question 4ciii: Interpolate

Finally perform interpolation by creating a view called `likely_data`. Your table header should look like:

| id | time | value | run | run_start | next_val | run_end | run_rank | run_size | interpolated |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |

In [99]:
grading_util.run_sql("""
SELECT id, time, value, run_start, run_end, run_rank, run_size
FROM backward
WHERE value IS NULL
LIMIT 10;
""")

,id,time,value,run_start,run_end,run_rank,run_size
0,a3d47b1a-985e-5395-a6ee-719dad9b580f,2018-06-13 09:00:00+00:00,None,0.056,0.056,1,2
1,a3e8e405-0eed-59b6-8747-fe892a6f93de,2018-06-09 04:00:00+00:00,None,281.600,280.800,1,2
2,a46fb790-028c-5d17-a3e6-7d08daec0c03,2018-06-13 09:00:00+00:00,None,281.390,281.460,1,2
3,a470ac6d-d448-522b-9b74-8ed27f24de25,2018-06-09 04:00:00+00:00,None,0.006,0.004,1,2
4,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 00:30:00+00:00,None,30.000,27.273,1,2
5,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 01:15:00+00:00,None,23.077,27.273,1,2
6,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 17:30:00+00:00,None,25.000,21.429,1,2
7,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 18:30:00+00:00,None,30.000,25.000,1,2
8,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 19:00:00+00:00,None,25.000,20.000,1,2
9,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 19:30:00+00:00,None,20.000,23.077,1,2


In [100]:
grading_util.run_sql("""
SELECT *
FROM backward
WHERE run_end = run_start AND value IS NULL
LIMIT 5;
""")


,id,time,value,run,run_start,next_val,run_end,run_rank,run_size
0,a3d47b1a-985e-5395-a6ee-719dad9b580f,2018-06-13 09:00:00+00:00,None,613,0.056,0.056,0.056,1,2
1,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-08 05:15:00+00:00,None,97,30.000,30.000,30.000,1,2
2,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-10 18:00:00+00:00,None,302,27.273,27.273,27.273,1,2
3,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-10 18:30:00+00:00,None,303,27.273,27.273,27.273,1,2
4,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-10 23:15:00+00:00,None,315,30.000,30.000,30.000,1,2


In [101]:
grading_util.run_sql("""
SELECT *
FROM backward
WHERE run_size = 0 AND value IS NULL
LIMIT 5;
""")


""


In [102]:
result_4ciii = grading_util.run_file("4ciii")[2]
grading_util.save_results("result_4ciii", result_4ciii)
result_4ciii

,id,time,value,run,run_start,next_val,run_end,run_rank,run_size,interpolated
0,a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 10:00:02+00:00,0.000,137,0.000,NaN,0.000,0,3,0.000000
1,a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 10:15:00+00:00,NaN,137,0.000,NaN,0.000,1,3,0.000000
2,a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 10:30:00+00:00,NaN,137,0.000,0.000,0.000,2,3,0.000000
3,a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 19:45:03+00:00,0.000,174,0.000,NaN,0.000,0,3,0.000000
4,a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 20:00:00+00:00,NaN,174,0.000,NaN,0.000,1,3,0.000000
...,...,...,...,...,...,...,...,...,...,...
95,a51d3bed-a8b6-5321-96a1-c5542caca70c,2018-06-07 22:15:00+00:00,NaN,20,5.825,76.923,76.923,2,3,53.223667
96,a51d3bed-a8b6-5321-96a1-c5542caca70c,2018-06-07 23:00:00+00:00,8.021,23,7.937,NaN,8.021,0,3,8.021000
97,a51d3bed-a8b6-5321-96a1-c5542caca70c,2018-06-07 23:15:00+00:00,NaN,23,8.021,NaN,103.448,1,3,39.830000
98,a51d3bed-a8b6-5321-96a1-c5542caca70c,2018-06-07 23:30:00+00:00,NaN,23,8.021,103.448,103.448,2,3,71.639000


In [103]:
result_4ciii = grading_util.load_results("result_4ciii")[0]

In [104]:
grader.check("q4ciii")

q4ciii results: All test cases passed!

## Question 5: Granularity Transforms
In this question, we will write a roll-up query on an ontology. This requires a bit of background explanation.

### Background: You MUST read this!

#### The Brick Ontology

An [ontology](https://en.wikipedia.org/wiki/Ontology_engineering) is a way of showing/mapping the properties and concepts of a subject area and how they are related. This is done by defining a set of terms and relational expressions that represent the entities in that subject area. 

The [Software-Defined Buildings](http://sdb.cs.berkeley.edu/sdb/) research project at Berkeley has led the development of a standard ontology for building metadata called [Brick](https://docs.brickschema.org/intro.html) that is getting a fair bit of attention in the world of IoT. Like many ontologies, it is represented as triples `(subject, predicate, object)`. In our database, the Brick ontology has been stored in a table called `ontology`.

#### The `subClassOf` Predicate and the `Sensor` Class

We are interested in readings from different classes of sensor devices. More specifically, we are interested in rows from the `metadata` table whose `metadata.class` field (representing the sensor class) maps to an `ontology` subject $s$, and that subject is in an ontology tuple ($s$, `http://www.w3.org/2000/01/rdf-schema#subClassOf`, `https://brickschema.org/schema/Brick#Sensor`).

Recall that $s$ is the subject, `...subClassOf` is the predicate, and `...Sensor` is the object. Thus, **the way to interpret this tuple in plain English is: "$s$ is a subclass of `Sensor`".**

Note: None of the URLs representing a `subject` or `object` work anymore, that is fine. Just focus on the last part of the URL, like `subClassOf` in the URL `http://www.w3.org/2000/01/rdf-schema#subClassOf`.

#### Sensor Diagram

The diagram below shows a few of the `subject`s and `object`s from `ontology` in ovals. (**Note that the diagram is incomplete, it does not display all types of sensors!**) The diagram is an example of a [directed graph](https://en.wikipedia.org/wiki/Directed_graph), a special type of data structure made of vertices and edges. In this case, the vertices are the `subject`s and `object`s, and the arrows represented directed edges from a `subject` to an `object`. **Everything you need to know about graphs will be included in the instructions.**

There is a black arrow between two ovals if there is a corresponding row in `ontology`. The immediate sub-classes of `Sensor` in the diagram are shown in yellow; we'll call them "`Sensor` children". Thus, a **"`Sensor` child"** is a sensor that is a **direct subclass** of `Sensor`(so `Sensor` is its parent).

All of the children underneath `Sensor` and `Sensor's` children are **transitive sensor children**. Transitive sensor children are shown in the diagram with aquamarine arrows. Note that direct sensor children are also transitive sensor children, and that the `ontology` table does NOT include transitive relationships (except from a direct subclass of `Sensor` to `Sensor`).

For example, on the left side of the diagram, `CO_Sensor` is a child of `Particulate_Matter_Sensor`, which is a child of `Sensor`. Thus, `CO_Sensor` is a transitive child of `Sensor`. (This is similar to the idea of inheritance in object-oriented programming!)

Note that unlike other trees that typically go from parent node down to leaf node, we are traversing from the child node **up** to the parent node.

<img src="data/subClass.png">

This image is available as `data/subClass.png`, and can be made available full-screen for ease of access.

#### The `transitive_subClassOf` Relation

The `transitive_subClassOf` materialized view is the set of *all* edges that compose the ontology graph, i.e., all black and aquamarine arrows in the diagram. It contains tuples of the form `(object, subject, hops, path)` where `subject` and `object` are connected transitively in `ontology` via one or more `subClassOf` predicates described above. `path` is a Postgres array type that shows the transitive path of class names through the ontology from subject to object, and `hops` is the length of that path. Run the next cell to see the first four rows of `transitive_subClassOf`:

In [105]:
grading_util.run_sql("SELECT * FROM transitive_subClassOf LIMIT 4;")

,object,subject,hops,path
0,https://brickschema.org/schema/Brick#Room,https://brickschema.org/schema/Brick#Ablutions_Room,1,"[https://brickschema.org/schema/Brick#Ablutions_Room, https://brickschema.org/schema/Brick#Room]"
1,ub1bL18C21,https://brickschema.org/schema/Brick#Ablutions_Room,1,"[https://brickschema.org/schema/Brick#Ablutions_Room, ub1bL18C21]"
2,https://brickschema.org/schema/Brick#Chiller,https://brickschema.org/schema/Brick#Absorption_Chiller,1,"[https://brickschema.org/schema/Brick#Absorption_Chiller, https://brickschema.org/schema/Brick#Chiller]"
3,ub1bL51C21,https://brickschema.org/schema/Brick#Absorption_Chiller,1,"[https://brickschema.org/schema/Brick#Absorption_Chiller, ub1bL51C21]"


We have already materialized this view for you, but to clarify its structure, we formed transitive “chains” by joining the ontology with itself. For example, we *could* have made the materialized view `transitive_2edge_subClassOf` with the below command. (Note that this materialized view does **not** exist).

```sql
CREATE MATERIALIZED VIEW transitive_2edge_subClassOf AS
SELECT o1.subject, o2.object 
FROM ontology AS o1
INNER JOIN ontology AS o2 ON o1.object = o2.subject
WHERE o1.predicate = 'http://www.w3.org/2000/01/rdf-schema#subClassOf'
    AND o2.predicate = 'http://www.w3.org/2000/01/rdf-schema#subClassOf';
``` 

Extending this example, in order to compute paths of length 3 we would have needed to join `ontology` on itself 2 times (thus you reference `ontology` 3 times total), and so on. To form all chains of arbitrary lengths requires the use of a recursive query. Again, we have already done this for you in the materialized view `transitive_subClassOf` that provides the result of that recursive query. 

*Just for fun: If you're curious about the recursive query that computes this view, you can issue the command `\d+ transitive_subClassOf` to Postgres. You may also want to read the documentation for SQL's [WITH RECURSIVE](https://www.postgresql.org/docs/9.1/queries-with.html) clause as implemented in Postgres.*

### Question 5a

We want to check the graph properties of the `subClassOf` predicate. It would be confusing if the `subClassOf` predicate had cycles! (A [cycle](https://en.wikipedia.org/wiki/Cycle_(graph_theory)) is a path in a graph where the start vertex is the same as the end vertex.)

Write a query on `transitive_subClassOf` to check for cycles. What property in `transitive_subClassOf` would be indicative of a cycle? Your query should return one row of one boolean column: `TRUE` if the predicate has cycles and `FALSE` otherwise.

**Hint:** You may find `EXISTS` to be useful here.

Your table header should look like this:

| cycle_exists |
| :--- |

In [106]:
result_5a = grading_util.run_file("5a")
grading_util.save_results("result_5a", result_5a)
result_5a

,cycle_exists
0,False


In [107]:
result_5a = grading_util.load_results("result_5a")[0]

In [108]:
grader.check("q5a")

q5a results: All test cases passed!

### Question 5b

Assuming it's not cyclic, the next question is whether the `subClassOf` predicate forms *tree-shaped* connections only. A *tree* is a special type of graph where each vertex has at most one outbound edge that points to its parent vertex. If a vertex has multiple outbound edges to its parent, then it is no longer a tree and is a more general type of graph, called a *directed acyclic graph*, or DAG.

In other words, we are looking to see if each subject is in a `subClassOf` predicate *with at most one object* (the single parent in the tree). 

Write a query that returns `TRUE` if *each* subject is in a `subClassOf` predicate with at most one object, and `FALSE` otherwise.

**Hint:** Refer to how we created `transitive_2edge_subClassOf` in the tutorial to see how we check if something is in a `subClassOf` predicate, specifically look at the `WHERE` clause.

Your table header should look like this:

| is_tree |
| :--- |

In [109]:
result_5b = grading_util.run_file("5b")
grading_util.save_results("result_5b", result_5b)
result_5b

,is_tree
0,False


In [110]:
result_5b = grading_util.load_results("result_5b")[0]

In [111]:
grader.check("q5b")

q5b results: All test cases passed!

### Question 5c

Now that we understand the graph properties of the ontology, let's use the ontology to do a roll-up. Remember from the lecture on granularity that a roll-up is a transformation to a coarser grain (e.g., go up in a hierarchy).

We're interested in the number of unique sensor `id`s from `metadata` that are transitively subclasses of each "`Sensor` child" class. To compute this, you will have to associate each `metadata.id` with a matching `mapping.brickclass` *(if there is one!)* by joining all the necessary tables together. (See the starter code.)

**YOUR TASK**:

1. Write a query that returns tuples of the form `(sensor_child, count)` that returns, for each "`Sensor` child" (the yellow nodes in the diagram above), the count of **distinct** `metadata.id` entries that are transitive subclasses of that "`Sensor` child" class.
2. Only include tuples from `metadata` that have a matching `brickclass`.
3. Order by `sensor_child` ascending.

**Hints:**

- It may be helpful to first create a CTE to get the unique "`Sensor` children" using information obtainable from the `ontology` table.
- Refer to `data/schema.png` for a refresher on the schema *and* explore the schema and data values in `metadata`, `mapping`, and `transitive_subClassOf` to determine how to join the tables together. Things to look for:
    - For each row in the `metadata` table, how do we match it with its correct sensor class (`mappings.brickclass`)?
    - Using your CTE of unique "`Sensor` children", `transitive_subClassOf`, and `mappings`, how can you determine when the current `transitive_subClassOf.subject` is a subclass of the current "`Sensor` child"?

Here is an example output table (**this is not the actual answer**):

| sensor_child | count |
| :--- | :--- |
| https://brickschema.org/schema/Brick#Energy_Sensor | 50 |
| https://brickschema.org/schema/Brick#Frequency_Sensor | 20 |

This output would indicate that `Energy_Sensor` is a a direct child of `Sensor`, and that it has 50 transitive children. Similarly, it would indicate that `Frequency_Sensor` is a direct child of `Sensor`, and that it has 20 transitive children.

In [112]:
grading_util.run_sql("SELECT * FROM mapping LIMIT 5;")



,rawname,brickclass
0,225 KVA Xfmr - Panel LNDB-2 Demand (kW),https://brickschema.org/schema/Brick#Active_Power_Sensor
1,Demand (kW),https://brickschema.org/schema/Brick#Active_Power_Sensor
2,Electric MB01 Demand (kW),https://brickschema.org/schema/Brick#Active_Power_Sensor
3,Electric MB02 Demand (kW),https://brickschema.org/schema/Brick#Active_Power_Sensor
4,Fire Pump Meter Demand (kW),https://brickschema.org/schema/Brick#Active_Power_Sensor


In [113]:
grading_util.run_sql("SELECT * FROM transitive_subClassOf WHERE object = 'https://brickschema.org/schema/Brick#Sensor' LIMIT 5;")


,object,subject,hops,path
0,https://brickschema.org/schema/Brick#Sensor,https://brickschema.org/schema/Brick#Adjust_Sensor,1,"[https://brickschema.org/schema/Brick#Adjust_Sensor, https://brickschema.org/schema/Brick#Sensor]"
1,https://brickschema.org/schema/Brick#Sensor,https://brickschema.org/schema/Brick#Air_Grains_Sensor,1,"[https://brickschema.org/schema/Brick#Air_Grains_Sensor, https://brickschema.org/schema/Brick#Sensor]"
2,https://brickschema.org/schema/Brick#Sensor,https://brickschema.org/schema/Brick#Angle_Sensor,1,"[https://brickschema.org/schema/Brick#Angle_Sensor, https://brickschema.org/schema/Brick#Sensor]"
3,https://brickschema.org/schema/Brick#Sensor,https://brickschema.org/schema/Brick#Capacity_Sensor,1,"[https://brickschema.org/schema/Brick#Capacity_Sensor, https://brickschema.org/schema/Brick#Sensor]"
4,https://brickschema.org/schema/Brick#Sensor,https://brickschema.org/schema/Brick#Conductivity_Sensor,1,"[https://brickschema.org/schema/Brick#Conductivity_Sensor, https://brickschema.org/schema/Brick#Sensor]"


In [114]:
grading_util.run_sql("SELECT COUNT(*) FROM metadata;")


,count
0,9509


In [115]:
result_5c = grading_util.run_file("5c")
grading_util.save_results("result_5c", result_5c)
result_5c

,sensor_child,count
0,https://brickschema.org/schema/Brick#Demand_Sensor,92
1,https://brickschema.org/schema/Brick#Power_Sensor,3106


In [116]:
result_5c = grading_util.load_results("result_5c")[0]

In [117]:
grader.check("q5c")

q5c results: All test cases passed!

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Congratulations! You have finished Project 3.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to export your written responses to `proj3.pdf`. We will not be accepting regrade requests for failure to render written responses.

**For your submission on Gradescope, you will only need to submit the single `proj3.zip` file generated by the export cell.** Please ensure that your submission `proj3.zip` file includes `proj3.pdf`, `proj3.ipynb`, and `queries.zip`. 

**Please ensure that public tests pass upon submission.** It is your responsibility to wait until the autograder finishes running. We will not be accepting regrade requests for submission issues.

**Common submission issues:** You MUST submit the generated zip file to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 3 directory within JupyterHub (remove `proj3.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

In [118]:
grading_util.prepare_submission_and_cleanup()

Closed grading database connection.
Created queries.zip
Created results.zip


---

To double-check your work, the cell below will rerun all of the autograder tests.

In [119]:
grader.check_all()

q1b results: All test cases passed!

q1c results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q3e results: All test cases passed!

q4a results: All test cases passed!

q4b results: All test cases passed!

q4ci results: All test cases passed!

q4cii results: All test cases passed!

q4ciii results: All test cases passed!

q5a results: All test cases passed!

q5b results: All test cases passed!

q5c results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [120]:
# Save your notebook first, then run this cell to export your submission.
grader.export(files=['queries.zip', 'results.zip'])